In [2]:
# Take magnitude as the feature
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


# Load dataset
measurement = np.load('../../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:8000]
n_comp = 4

# Train-test split
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print(f'trainData - {trainCIR.shape}')
print(f'testData - {testCIR.shape}')

# Define channels
alice_channel = 3  # A -> B (legitimate)
eve_channel = 6  # E -> B (illegitimate)


def apply_pca(data, n_components):
    # data: (samples, 251)
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)  # (samples, 251)
    
    pca = PCA(n_components=n_components)
    data_pca = pca.fit_transform(data_scaled)  # (samples, n_components)
    
    return data_pca, scaler, pca

# ----------------- Train -----------------
# Feature Extraction
alice_train_CIRs = trainCIR[:, alice_channel, :, :]  # (6400, 251, 2)
eve_train_CIRs = trainCIR[:, eve_channel, :, :]      # (6400, 251, 2)


# alice_train_magnitude = np.linalg.norm(alice_train_CIRs, axis=2) 
alice_train_magnitude = np.abs(alice_train_CIRs[..., 0] + 1j * alice_train_CIRs[..., 1])  # (6400, 251)
eve_train_magnitude = np.abs(eve_train_CIRs[..., 0] + 1j * eve_train_CIRs[..., 1])        # (6400, 251)
train_cirs = np.vstack((alice_train_magnitude, eve_train_magnitude))  # (12800, 251)


# Apply PCA
train_cirs_pca, scaler, pca = apply_pca(train_cirs, n_components=n_comp)  # (12800, n_components)


# Labels
alice_train_labels = np.zeros(alice_train_CIRs.shape[0])  # '0' for Alice
eve_train_labels = np.ones(eve_train_CIRs.shape[0])       # '1' for Eve
train_labels = np.hstack((alice_train_labels, eve_train_labels))  # (12800,)

# ----------------- Test -----------------
# Feature Extraction
alice_test_CIRs = testCIR[:, alice_channel, :, :]  # (1600, 251, 2)
eve_test_CIRs = testCIR[:, eve_channel, :, :]      # (1600, 251, 2)


alice_test_magnitude = np.abs(alice_test_CIRs[..., 0] + 1j * alice_test_CIRs[..., 1])  # (1600, 251)
eve_test_magnitude = np.abs(eve_test_CIRs[..., 0] + 1j * eve_test_CIRs[..., 1])        # (1600, 251)
test_cirs = np.vstack((alice_test_magnitude, eve_test_magnitude))  # (3200, 251)



test_cirs_scaled = scaler.transform(test_cirs)  # (3200, 251)
test_cirs_pca = pca.transform(test_cirs_scaled)  # (3200, n_components)


# Labels
alice_test_labels = np.zeros(alice_test_CIRs.shape[0])  # '0' for Alice
eve_test_labels = np.ones(eve_test_CIRs.shape[0])       # '1' for Eve
test_labels = np.hstack((alice_test_labels, eve_test_labels))  # (3200,)

# ----------------- Classification -----------------
# KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(train_cirs_pca, train_labels)

# Predict
predictions = knn.predict(test_cirs_pca)

# ----------------- Evaluation -----------------
# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0

# False Alarm Rate (FAR)
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0

# Gamma calculation
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0

# Authentication Rate (AR)
denominator = (tp + fn) + gamma * (tn + fp)
AR = (tp + gamma * tn) / denominator if denominator > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


trainData - (6400, 15, 251, 2)
testData - (1600, 15, 251, 2)
Classification Accuracy: 56.75%
tp: 925
tn: 891
fp: 709
fn: 675
MDR: 0.443125
FAR: 0.421875
AR: 0.5675
